In [1]:
import os
import numpy as np
from PIL import Image, UnidentifiedImageError #PIL is python image library
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2024-09-04 16:39:49.058782: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Function to load images from a directory and label them
def load_images_from_folder(folder):
    images = []
    labels = []
    class_names = sorted([d for d in os.listdir(folder) if os.path.isdir(os.path.join(folder, d))])  # Filter to only directories
    for label, class_name in enumerate(class_names):
        class_folder = os.path.join(folder, class_name)
        for filename in os.listdir(class_folder):
            img_path = os.path.join(class_folder, filename)
            try:
                # Open the image file and convert to RGB
                with Image.open(img_path) as img:
                    img = img.convert('RGB')  # Ensure all images are RGB
                    img = img.resize((100, 100))
                    img_array = np.array(img)
                    images.append(img_array)
                    labels.append(label)
            except (IOError, UnidentifiedImageError) as e:
                # Skip files that can't be identified as images
                print(f"Skipping file {img_path}: {e}")
    return np.array(images), np.array(labels)

# Load training and testing data
train_folder = 'multiclassCNN/train'
test_folder = 'multiclassCNN/test'

x_train, y_train = load_images_from_folder(train_folder)
x_test, y_test = load_images_from_folder(test_folder)

# Normalize the data
x_train = x_train / 255.0
x_test = x_test / 255.0

In [3]:

# Convert labels to categorical (one-hot encoding)
num_classes = len(np.unique(y_train))
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [4]:
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(100, 100, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    Dropout(0.2),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax') 
])


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:

# Define callbacks
#early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
#lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)


In [6]:

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [7]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(datagen.flow(x_train, y_train, batch_size=32), epochs=10, validation_data=(x_test, y_test))  # , callbacks=[early_stopping, lr_scheduler]


Epoch 1/10


/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


229/229 ━━━━━━━━━━━━━━━━━━━━ 85s 339ms/step - accuracy: 0.1098 - loss: 2.6880 - val_accuracy: 0.2097 - val_loss: 2.4901
Epoch 2/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 71s 304ms/step - accuracy: 0.1981 - loss: 2.4806 - val_accuracy: 0.2629 - val_loss: 2.3298
Epoch 3/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 81s 349ms/step - accuracy: 0.2584 - loss: 2.3074 - val_accuracy: 0.2971 - val_loss: 2.2072
Epoch 4/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 81s 346ms/step - accuracy: 0.2915 - loss: 2.2001 - val_accuracy: 0.3259 - val_loss: 2.1160
Epoch 5/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 83s 354ms/step - accuracy: 0.3229 - loss: 2.1787 - val_accuracy: 0.3444 - val_loss: 2.0748
Epoch 6/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 82s 351ms/step - accuracy: 0.3364 - loss: 2.1146 - val_accuracy: 0.3770 - val_loss: 2.0192
Epoch 7/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 88s 376ms/step - accuracy: 0.3630 - loss: 2.0641 - val_accuracy: 0.3639 - val_loss: 1.9735
Epoch 8/10
 36/229 ━━━━━━━━━━━━━━━━━━━━ 1:05 338ms/step - accuracy: 0.3322 - loss: 2.1066

In [ ]:

# Evaluate the model
model.evaluate(x_test, y_test)


In [ ]:
# Display a random image and its prediction
idx2 = random.randint(0, len(x_test) - 1)
plt.imshow(x_test[idx2])
plt.show()

# Predict the class of the image
y_pred = model.predict(x_test[idx2, :].reshape(1, 100, 100, 3))
predicted_class_idx = np.argmax(y_pred)

# Get the class names from the folder structure
class_names = sorted([d for d in os.listdir(train_folder) if os.path.isdir(os.path.join(train_folder, d))])

# Map the predicted class index and actual class index to their respective class names
predicted_class_name = class_names[predicted_class_idx]
actual_class_name = class_names[np.argmax(y_test[idx2])]

print(f"Predicted Class: {predicted_class_name}, Actual Class: {actual_class_name}")
